#### task:

given gene expression (RNA), predict surface protein levels.

- train/test_cite_inputs.h5: Rows corresponding to cells and columns corresponding to genes given by {gene_name}_{gene_ensemble-ids}\
 - have 22085 genes
 - from these: 84 are handselected and hstacked to the right
 - ... and on the remaining 22001 genes, TruncatedSVD is applied -> 128 svd components instead -> hstacked to the left
- train_cite_targets.h5: Surface protein levels for the same cells
 - 140 proteins, e.g. CD36

previous notebooks: compute shap values 
- on svd components + handselected: get shap values of shape (140, n_samples, n_svd_components+84)
- backpropagated attributions to all genes: get attr_all_22085_genes of shape (140, n_samples, 22085)
- => then only used shap_values[0] or attr_all_22085_genes[0]
- now: handle all 140 classes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import pandas as pd
import os

import shap

import anndata as ad
import scipy
import scanpy as sc

from scipy import stats


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [3]:
os.chdir('../..')

In [4]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/'
index_path = lrz_path + 'open-problems-multimodal-3rd-solution/input/preprocess/cite/'
train_column = np.load(index_path + "train_cite_raw_inputs_idxcol.npz", allow_pickle=True)["columns"]

In [5]:
def get_top_features(shap_values, xtest, top_n):
    shap_sum = np.abs(shap_values).sum(axis=0)
    top_features_indices = np.argsort(shap_sum)[::-1][:top_n]  # Get the indices of the top n features
    top_feature_names_shap = xtest.var_names[top_features_indices]
    return top_feature_names_shap

In [6]:
def get_common_elements(attr, xtest, top_features_considered, min_percentage):
    element_counts = {}

    for i in range(140):
        for elt in get_top_features(attr[i], xtest, top_features_considered):
            element_counts[elt] = element_counts.get(elt, 0) + 1

    # Get the elements that appear in at least min_percentage of the lists
    common_elements = {element for element, count in element_counts.items() if count / 140 >= min_percentage}
    return common_elements

### attr_all_22085_genes of model 16

In [7]:
proteins = np.load('4.model/pred/proteins.npy', allow_pickle=True)
proteins

array(['CD86', 'CD274', 'CD270', 'CD155', 'CD112', 'CD47', 'CD48', 'CD40',
       'CD154', 'CD52', 'CD3', 'CD8', 'CD56', 'CD19', 'CD33', 'CD11c',
       'HLA-A-B-C', 'CD45RA', 'CD123', 'CD7', 'CD105', 'CD49f', 'CD194',
       'CD4', 'CD44', 'CD14', 'CD16', 'CD25', 'CD45RO', 'CD279', 'TIGIT',
       'Mouse-IgG1', 'Mouse-IgG2a', 'Mouse-IgG2b', 'Rat-IgG2b', 'CD20',
       'CD335', 'CD31', 'Podoplanin', 'CD146', 'IgM', 'CD5', 'CD195',
       'CD32', 'CD196', 'CD185', 'CD103', 'CD69', 'CD62L', 'CD161',
       'CD152', 'CD223', 'KLRG1', 'CD27', 'CD107a', 'CD95', 'CD134',
       'HLA-DR', 'CD1c', 'CD11b', 'CD64', 'CD141', 'CD1d', 'CD314',
       'CD35', 'CD57', 'CD272', 'CD278', 'CD58', 'CD39', 'CX3CR1', 'CD24',
       'CD21', 'CD11a', 'CD79b', 'CD244', 'CD169', 'integrinB7', 'CD268',
       'CD42b', 'CD54', 'CD62P', 'CD119', 'TCR', 'Rat-IgG1', 'Rat-IgG2a',
       'CD192', 'CD122', 'FceRIa', 'CD41', 'CD137', 'CD163', 'CD83',
       'CD124', 'CD13', 'CD2', 'CD226', 'CD29', 'CD303', 'CD49b', 'C

In [8]:
xtest_16 = ad.read_h5ad('4.model/pred/private_test_input_128_svd_50_samples.h5ad')
private_test_input_raw = pd.read_pickle('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/kaggle/full_data/20220830_citeseq_rna_count_test_input_private_raw.pkl')
sample_cells = np.array(xtest_16.obs_names)
xtest_all_genes = private_test_input_raw.loc[sample_cells]

In [9]:
attr_all_22085_genes_16 = pd.read_pickle(lrz_path + 'large_preprocessed_files/attr_all_genes/attr_all_22085_genes_16_50_samples_p_ct_distr.pkl')
for i in range(len(attr_all_22085_genes_16)):
    attr_all_22085_genes_16[i] = attr_all_22085_genes_16[i].reindex(columns=train_column)

In [10]:
top_20 = get_top_features(attr_all_22085_genes_16[0], ad.AnnData(xtest_all_genes, dtype=xtest_all_genes.dtypes), 20)
top_20

X converted to numpy array with dtype float64


Index(['ENSG00000135218_CD36', 'ENSG00000010278_CD9', 'ENSG00000100097_LGALS1',
       'ENSG00000183971_NPW', 'ENSG00000105472_CLEC11A',
       'ENSG00000184897_H1FX', 'ENSG00000100368_CSF2RB',
       'ENSG00000265972_TXNIP', 'ENSG00000122862_SRGN', 'ENSG00000105205_CLC',
       'ENSG00000236824_BCYRN1', 'ENSG00000125148_MT2A',
       'ENSG00000077984_CST7', 'ENSG00000245532_NEAT1', 'ENSG00000124766_SOX4',
       'ENSG00000244734_HBB', 'ENSG00000005381_MPO', 'ENSG00000101745_ANKRD12',
       'ENSG00000163191_S100A11', 'ENSG00000105383_CD33'],
      dtype='object', name='gene_ids')

In [11]:
get_top_features(attr_all_22085_genes_16[2], ad.AnnData(xtest_all_genes, dtype=xtest_all_genes.dtypes), 20)

Index(['ENSG00000135218_CD36', 'ENSG00000117091_CD48', 'ENSG00000010278_CD9',
       'ENSG00000184897_H1FX', 'ENSG00000183971_NPW',
       'ENSG00000105472_CLEC11A', 'ENSG00000124766_SOX4',
       'ENSG00000100097_LGALS1', 'ENSG00000244734_HBB',
       'ENSG00000101745_ANKRD12', 'ENSG00000236824_BCYRN1',
       'ENSG00000135655_USP15', 'ENSG00000125148_MT2A', 'ENSG00000196565_HBG2',
       'ENSG00000245532_NEAT1', 'ENSG00000265972_TXNIP', 'ENSG00000015285_WAS',
       'ENSG00000136997_MYC', 'ENSG00000048649_RSF1',
       'ENSG00000134283_PPHLN1'],
      dtype='object', name='gene_ids')

In [12]:
get_top_features(attr_all_22085_genes_16[3], ad.AnnData(xtest_all_genes, dtype=xtest_all_genes.dtypes), 20)

Index(['ENSG00000117091_CD48', 'ENSG00000010278_CD9', 'ENSG00000135218_CD36',
       'ENSG00000184897_H1FX', 'ENSG00000105369_CD79A', 'ENSG00000188404_SELL',
       'ENSG00000100097_LGALS1', 'ENSG00000124766_SOX4', 'ENSG00000183971_NPW',
       'ENSG00000101745_ANKRD12', 'ENSG00000105472_CLEC11A',
       'ENSG00000236824_BCYRN1', 'ENSG00000265972_TXNIP',
       'ENSG00000245532_NEAT1', 'ENSG00000135655_USP15',
       'ENSG00000125148_MT2A', 'ENSG00000114013_CD86', 'ENSG00000244734_HBB',
       'ENSG00000072274_TFRC', 'ENSG00000128322_IGLL1'],
      dtype='object', name='gene_ids')

In [13]:
# top attributing genes across all targets
get_common_elements(attr_all_22085_genes_16, ad.AnnData(xtest_all_genes, dtype=xtest_all_genes.dtypes), 20, 0.75)

{'ENSG00000100097_LGALS1',
 'ENSG00000101745_ANKRD12',
 'ENSG00000105472_CLEC11A',
 'ENSG00000124766_SOX4',
 'ENSG00000135218_CD36',
 'ENSG00000135655_USP15',
 'ENSG00000183971_NPW',
 'ENSG00000184897_H1FX',
 'ENSG00000236824_BCYRN1',
 'ENSG00000265972_TXNIP'}

### model 17:

In [14]:
xtest_17 = ad.read_h5ad('4.model/pred/private_test_input_64_svd_50_samples.h5ad')
attr_all_22085_genes_17 = pd.read_pickle(lrz_path + 'large_preprocessed_files/attr_all_genes/attr_all_22085_genes_17_50_samples_p_ct_distr.pkl')
for i in range(len(attr_all_22085_genes_17)):
    attr_all_22085_genes_17[i] = attr_all_22085_genes_17[i].reindex(columns=train_column)

### spearman correlation

In [15]:
attr_all_22085_genes_16[1]['ENSG00000139193_CD27'].unique()

array([ 0.        , -0.28077111, -0.14489721])

In [16]:
attr_all_22085_genes_17[1]['ENSG00000139193_CD27'].unique()

array([ 0.        , -0.00733932, -0.01350221])

In [17]:
gene_corr = {}
for col in attr_all_22085_genes_16[53].columns:
    res = stats.spearmanr(np.array(attr_all_22085_genes_16[53][col]), np.array(attr_all_22085_genes_17[53][col]))
    gene_corr[col] = res.statistic

In [18]:
sorted(gene_corr.items(), key=lambda x:x[1])[-50:]

[('ENSG00000198556_ZNF789', 0.3400536680765907),
 ('ENSG00000169504_CLIC4', 0.34138284510777345),
 ('ENSG00000130222_GADD45G', 0.3419717245505211),
 ('ENSG00000182054_IDH2', 0.344548072112542),
 ('ENSG00000128165_ADM2', 0.34877574510812326),
 ('ENSG00000184988_TMEM106A', 0.35276019745933784),
 ('ENSG00000165168_CYBB', 0.3535766239491168),
 ('ENSG00000091106_NLRC4', 0.35401492490317704),
 ('ENSG00000248323_LUCAT1', 0.3595964048686112),
 ('ENSG00000168685_IL7R', 0.3598295496289766),
 ('ENSG00000143674_MAP3K21', 0.35985725830998033),
 ('ENSG00000198682_PAPSS2', 0.36157323733254965),
 ('ENSG00000129824_RPS4Y1', 0.3637266776521791),
 ('ENSG00000099250_NRP1', 0.3648775675136706),
 ('ENSG00000179583_CIITA', 0.36746650993069335),
 ('ENSG00000147852_VLDLR', 0.36780936987240714),
 ('ENSG00000148680_HTR7', 0.37251508758672086),
 ('ENSG00000153823_PID1', 0.3734037245785097),
 ('ENSG00000167703_SLC43A2', 0.37747466160073606),
 ('ENSG00000263513_FAM72C', 0.3777467104687735),
 ('ENSG00000266028_SRGAP

- first protein:
ENSG00000139193_CD27 has correlation 1.0 \
-> all attribution values except for two are 0; this holds for both models 16 and 17 \
-> in both models this gene does not influence the prediction of the first protein CD86 \
- same for second protein: corr 0.999
- for 53rd protein (CD27): corr -0.999

In [19]:
for gene in gene_corr.keys():
    if 'CD27' in gene:
        print(gene_corr[gene])

-0.9999835797735651
0.11782444871270087
0.050413706001098554
0.08987730512085813


In [20]:
gene_corr['ENSG00000139193_CD27']

-0.9999835797735651

### quantify results from plotting.ipynb using kurtosis variance metric

In [21]:
attr_all_22085_genes_17[0]

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
0,0.000136,0.000431,0.000498,0.000939,0.000007,-0.000766,-0.003348,-0.000048,-0.000220,-0.000737,...,-0.000029,-0.002129,0.001480,0.002161,0.000029,0.000546,0.000202,0.001743,0.013976,0.001249
1,-0.000138,0.000377,0.001151,0.001888,0.000066,-0.002372,0.006538,0.001002,0.001100,0.002942,...,-0.001238,-0.004052,0.001034,-0.026512,-0.000150,0.000887,-0.000185,-0.004241,-0.045451,0.000368
2,-0.000077,-0.000343,0.001450,0.000222,-0.000016,-0.001436,0.001670,-0.004448,-0.000135,-0.000176,...,0.001031,-0.002540,0.000433,-0.002216,0.000216,-0.000044,-0.000339,-0.000997,0.002788,0.000761
3,-0.000043,-0.000274,0.001451,-0.000025,0.000014,-0.001380,-0.004225,-0.000628,0.000048,-0.000010,...,0.003288,0.000662,-0.000555,0.002060,0.000150,-0.000202,0.000293,0.000753,-0.005679,0.000670
4,0.000046,0.000877,0.001341,0.003048,0.000114,0.000155,0.002438,0.014132,0.002368,0.002059,...,-0.003785,-0.005271,0.000143,-0.034581,-0.000421,0.001652,0.002118,-0.003219,-0.040279,-0.001113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0.000039,0.000807,-0.000115,0.001784,0.000060,-0.001080,0.002503,0.008608,0.000706,0.002036,...,-0.002452,-0.003580,-0.000909,-0.020892,-0.000261,0.000789,0.001020,-0.002363,-0.030549,0.001514
346,0.000029,0.000102,0.000089,0.000073,0.000002,0.000013,-0.000200,0.001594,-0.000252,0.000316,...,0.000058,0.000159,0.000074,0.000127,0.000026,-0.000046,0.000638,-0.000264,0.004508,0.000477
347,0.000064,0.000574,0.000014,0.000790,0.000025,-0.000471,0.000997,0.007318,0.000089,0.000383,...,-0.001085,-0.003700,-0.001080,-0.014419,-0.000140,0.000373,0.001161,-0.001029,-0.010368,0.000633
348,-0.000038,0.000551,0.000051,0.001039,0.000043,-0.000636,0.002708,0.004894,0.000473,0.002150,...,-0.001585,-0.002055,0.000420,-0.013068,-0.000182,0.000575,0.000500,-0.002142,-0.024802,0.000253


In [22]:
stats.kurtosis(attr_all_22085_genes_17[121]['ENSG00000010278_CD9'])

-1.1157470558557832

In [23]:
attr_all_22085_genes_16[121]['ENSG00000010278_CD9'].min()

0.0

In [24]:
for top in get_top_features(attr_all_22085_genes_17[121], ad.AnnData(xtest_all_genes, dtype=xtest_all_genes.dtypes), 20):
    print(stats.iqr(attr_all_22085_genes_17[121][top]))

1.2859274340778954
0.9263992332530588
0.7555682334295706
0.632317845692077
0.5198712138926351
0.5287259468267762
0.5130606744916917
0.4988977894295687
0.49102977509995277
0.48631912689729706
0.4434940921255437
0.3922978111837599
0.3779115192144079
0.3757374655846847
0.38008467169157467
0.37678055434017854
0.3685003174403199
0.36528840813946
0.359598336465624
0.35029595815794734


X converted to numpy array with dtype float64


#### compute mean per cell type

In [63]:
def mean_per_cell_type(attr_all_genes, xtest, n_top, stat):
    ''' stat: pass mean, min, max, median '''
    top_feature_names_shap = get_top_features(attr_all_genes, xtest, n_top)
    # create df with shap values of top n features + cell type    
    shap_cell_types = {}
    # Add 'Cell Type' column
    shap_cell_types['Cell Type'] = xtest.obs['cell_type']
    # Add attribution values of top n features
    for feature in top_feature_names_shap:
        column_values = np.array(attr_all_genes)[:, xtest.var_names.get_loc(feature)]
        shap_cell_types[feature] = column_values

    # Create DataFrame
    shap_cell_types = pd.DataFrame(shap_cell_types)
    
    if stat == 'mean':
        mean_per_ct = shap_cell_types.groupby('Cell Type').apply(lambda x: x.abs().mean())
        mean_per_ct.reset_index(inplace=True)
        # Calculate the mean across all rows (excluding the 'cell type' column)
        overall_mean = shap_cell_types.drop('Cell Type', axis=1).abs().mean()
        # Add the overall mean as a new row to the DataFrame
        mean_per_ct.loc[len(mean_per_ct)] = ['Overall'] + overall_mean.tolist()
    elif stat == 'min':
        mean_per_ct = shap_cell_types.groupby('Cell Type').min()
        mean_per_ct.reset_index(inplace=True)
        # Calculate the mean across all rows (excluding the 'cell type' column)
        overall_mean = shap_cell_types.drop('Cell Type', axis=1).min()
        # Add the overall mean as a new row to the DataFrame
        mean_per_ct.loc[len(mean_per_ct)] = ['Overall'] + overall_mean.tolist()
    elif stat == 'max':
        mean_per_ct = shap_cell_types.groupby('Cell Type').max()
        mean_per_ct.reset_index(inplace=True)
        # Calculate the mean across all rows (excluding the 'cell type' column)
        overall_mean = shap_cell_types.drop('Cell Type', axis=1).max()
        # Add the overall mean as a new row to the DataFrame
        mean_per_ct.loc[len(mean_per_ct)] = ['Overall'] + overall_mean.tolist()
    elif stat == 'median':
        mean_per_ct = shap_cell_types.groupby('Cell Type').median()
        mean_per_ct.reset_index(inplace=True)
        # Calculate the mean across all rows (excluding the 'cell type' column)
        overall_mean = shap_cell_types.drop('Cell Type', axis=1).median()
        # Add the overall mean as a new row to the DataFrame
        mean_per_ct.loc[len(mean_per_ct)] = ['Overall'] + overall_mean.tolist()
    
    return mean_per_ct

In [65]:
mean_per_cell_type(attr_all_22085_genes_16[121], ad.AnnData(xtest_all_genes, obs=xtest_16.obs, dtype=xtest_all_genes.dtypes), 20, 'mean')

X converted to numpy array with dtype float64


,Cell Type,ENSG00000010278_CD9,ENSG00000135218_CD36,ENSG00000184897_H1FX,ENSG00000101745_ANKRD12,ENSG00000183971_NPW,ENSG00000124766_SOX4,ENSG00000135655_USP15,ENSG00000134283_PPHLN1,ENSG00000100097_LGALS1,...,ENSG00000048649_RSF1,ENSG00000123405_NFE2,ENSG00000197061_HIST1H4C,ENSG00000265972_TXNIP,ENSG00000153310_FAM49B,ENSG00000196565_HBG2,ENSG00000122779_TRIM24,ENSG00000102007_PLP2,ENSG00000102409_BEX4,ENSG00000120686_UFM1
0,BP,0.345274,0.018846,0.051737,0.039328,0.033687,0.036823,0.033996,0.033108,0.033051,...,0.031992,0.031016,0.025575,0.024983,0.029185,0.024222,0.028661,0.030007,0.027691,0.027269
1,EryP,0.136655,0.113294,0.048880,0.032755,0.024259,0.032750,0.032449,0.030773,0.027123,...,0.027026,0.028533,0.024920,0.029520,0.027053,0.028103,0.027073,0.023174,0.023872,0.025537
2,HSC,0.628575,0.003720,0.041991,0.029267,0.025890,0.027528,0.026406,0.026309,0.025042,...,0.024779,0.023181,0.023614,0.022091,0.023756,0.020323,0.022232,0.022526,0.020350,0.021006
3,MasP,0.488781,0.006003,0.074119,0.054235,0.049142,0.048645,0.050635,0.047042,0.042958,...,0.043520,0.040488,0.032411,0.035968,0.039833,0.034853,0.037774,0.036596,0.038255,0.037362
4,MkP,1.239405,0.151076,0.042048,0.029735,0.036852,0.028465,0.027854,0.024181,0.029674,...,0.024302,0.024047,0.037205,0.029547,0.021144,0.026031,0.022427,0.022470,0.023503,0.021102
5,MoP,0.905386,0.076862,0.035039,0.024205,0.034551,0.022506,0.024247,0.020820,0.021402,...,0.020553,0.019382,0.018438,0.026032,0.019703,0.024855,0.019872,0.019583,0.017846,0.018108
6,NeuP,0.120816,0.002167,0.042811,0.028581,0.032344,0.027798,0.028417,0.026207,0.024656,...,0.025707,0.024692,0.027806,0.019985,0.023581,0.024385,0.022459,0.021046,0.020644,0.021603
7,Overall,0.552128,0.053138,0.048089,0.034015,0.033818,0.032073,0.032001,0.029777,0.029129,...,0.028268,0.027334,0.027139,0.026875,0.026322,0.026110,0.025785,0.025057,0.024595,0.024570


In [66]:
mean_per_cell_type(attr_all_22085_genes_17[121], ad.AnnData(xtest_all_genes, obs=xtest_16.obs, dtype=xtest_all_genes.dtypes), 20, 'mean')

X converted to numpy array with dtype float64


,Cell Type,ENSG00000010278_CD9,ENSG00000265972_TXNIP,ENSG00000183971_NPW,ENSG00000245532_NEAT1,ENSG00000094916_CBX5,ENSG00000136997_MYC,ENSG00000044574_HSPA5,ENSG00000143013_LMO4,ENSG00000078596_ITM2A,...,ENSG00000167552_TUBA1A,ENSG00000102409_BEX4,ENSG00000122033_MTIF3,ENSG00000196565_HBG2,ENSG00000178057_NDUFAF3,ENSG00000204482_LST1,ENSG00000106211_HSPB1,ENSG00000145734_BDP1,ENSG00000124575_HIST1H1D,ENSG00000125148_MT2A
0,BP,0.293941,0.231850,0.198248,0.155329,0.130741,0.126863,0.125929,0.122794,0.117860,...,0.112670,0.098918,0.106017,0.091842,0.098702,0.098096,0.089925,0.090540,0.090772,0.093812
1,EryP,0.124157,0.129876,0.086183,0.079545,0.060917,0.059710,0.055760,0.063996,0.053548,...,0.053986,0.044191,0.057913,0.046473,0.051913,0.048726,0.042566,0.040312,0.047756,0.051388
2,HSC,0.568469,0.430292,0.341437,0.285482,0.233227,0.238491,0.227086,0.225480,0.222682,...,0.203747,0.197696,0.172369,0.181114,0.173962,0.167285,0.171668,0.168320,0.162547,0.162312
3,MasP,0.434178,0.326899,0.260674,0.212424,0.177440,0.186249,0.177976,0.172126,0.170661,...,0.160300,0.140823,0.137024,0.139454,0.134918,0.134517,0.132241,0.126937,0.123228,0.125258
4,MkP,1.236105,0.895884,0.744280,0.609649,0.497699,0.495432,0.492131,0.480927,0.473684,...,0.417701,0.368282,0.374562,0.363765,0.371411,0.356967,0.360712,0.349660,0.346803,0.329207
5,MoP,0.791599,0.594043,0.474342,0.388421,0.328184,0.322209,0.317058,0.303555,0.301773,...,0.285850,0.265025,0.238529,0.243680,0.247346,0.231643,0.230466,0.235372,0.218919,0.227178
6,NeuP,0.111354,0.102498,0.074488,0.072055,0.060078,0.057457,0.056074,0.055178,0.049452,...,0.047724,0.065103,0.042809,0.051863,0.039454,0.042620,0.041127,0.039202,0.047130,0.038525
7,Overall,0.508543,0.387334,0.311379,0.257558,0.212612,0.212344,0.207430,0.203437,0.198523,...,0.183140,0.168577,0.161317,0.159742,0.159672,0.154265,0.152672,0.150049,0.148165,0.146811


### ignore below

### cluster the top features of all 140 attr_all_22085_genes[i]

In [ ]:
lists = [get_top_features(attr_all_22085_genes[i], xtest_all_genes, 20) for i in range(140)]

In [ ]:
from sklearn.cluster import KMeans

similarity_matrix = np.zeros((len(lists), len(lists)))
for i in range(len(lists)):
    for j in range(i+1, len(lists)):
        common_elements = len(set(lists[i]).intersection(lists[j]))
        similarity_matrix[i, j] = common_elements
        similarity_matrix[j, i] = common_elements

# Apply K-means clustering
n_clusters = 4  # Number of clusters to create
kmeans = KMeans(n_clusters=n_clusters)
cluster_labels = kmeans.fit_predict(similarity_matrix)

# Print the cluster labels
for i, label in enumerate(cluster_labels):
    if i < 5:
        print(f"List {i}: Cluster {label}")

In [ ]:
from sklearn.metrics import silhouette_score

# Calculate Silhouette Score
# score between -1 and 1, where higher values indicate better-defined and well-separated clusters.
silhouette_avg = silhouette_score(similarity_matrix, cluster_labels)
# highest for 3 or 4 clusters
print("Silhouette Score:", silhouette_avg)

In [ ]:
sc.pp.neighbors(xtest, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.leiden(xtest_all_genes)

In [ ]:
sc.tl.paga(xtest_all_genes)
sc.pl.paga(xtest_all_genes, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(xtest_all_genes, init_pos='paga')

In [ ]:
sc.tl.umap(xtest_all_genes)
sc.pl.umap(xtest_all_genes, color=['ENSG00000135218_CD36', 'ENSG00000072274_TFRC', 'ENSG00000184897_H1FX'])


In [ ]:
sc.tl.leiden(xtest_all_genes)

In [ ]:
sc.pl.umap(xtest_all_genes, color=['leiden', 'ENSG00000135218_CD36', 'ENSG00000072274_TFRC'])